# Assignment 4: Named entity recognition
Create a model for Named Entity Recognition for dataset CoNLL 2002.
Your quality metric = f1_macro

In your solution you should use: RandomForest, Gradient Boosting (xgboost, lightgbm, catboost)
Tutorials:
* https://github.com/Microsoft/LightGBM/tree/master/examples/python-guide
* https://github.com/catboost/tutorials

More baselines you beat - better your score

* baseline 1 [3 points]: 0.0604 random labels
* baseline 2 [5 points]: 0.3966 PoS features + logistic regression
* baseline 3 [8 points]: 0.8122 word2vec cbow embedding + baseline 2 + svm

[1 point] using feature engineering (creating features not presented in the baselines)

! Your results must be reproducible. You should explicitly set all seeds random_states in yout model.
! Remember to use proper training pipeline.

bonus, think about:

[1 point] Why did we select f1 score with macro averaging as our classification quality measure? What other metrics are suitable?

# Ответ на дополнительный вопрос

Задачу распознавания именованных сущностей можно представить как задачу многоклассовой классификации. Один из классов определяется отрицательно, сюда относятся токены, которые не являются именованными сущностями, то есть их нельзя приписать никакому другому классу. Очевидно, что в текстах на любом естественном языке доля таких токенов обычно значительно больше доли токенов, которые входят в состав какой-либо именованной сущности.

Отсюда следует, например, что метрика accuracy не подходит для задач NER: она недостаточно наказывает за false negative предсказания. Иными словами, модель, которая любой токен помечает как не-сущность, покажет довольно неплохие результаты по метрике accuracy. К тому же, неполное распознавание именованной сущности (e. g., только имя, хотя нужно «имя + фамилия») недостаточно сильно штрафуется. По схожим причинам обычно не используется AUC-ROC.

При оценке работы моделей, распознающих именованные сущности, довольно часто используют F1-метрику. Основное достоинство F1-метрики состоит в том, что она не зависит от объёма класса true negatives (не-сущности), который в задачах NER типично очень большой. Это позволяет учитывать способность выделять именованные сущности на фоне большого количества токенов, которые ими не являются, не страдая проблемой недостаточного предсказания.

Помимо F1-метрики, можно также использовать по отдельности precision и recall, на расчёт которых опирается F1-метрика. Конечно, можно воспользоваться и AUC-PR. На данный момент разрабатываются и другие методы оценки качества работы NER-алгоритмов, например, token & separator model (Esuli & Sebastiani, 2010).

# Решение задания

В работе использован алгоритм Word2Vec CBoW, RandomForestClassifier. Feature engineering не проводился.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


SEED=1337

In [2]:
df = pd.read_csv("../input/ner_short.csv")
df.head()

,Unnamed: 0,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,sentence_idx,word,tag
0,0,NNS,demonstrators,IN,of,NNS,__START1__,__START2__,__START2__,__START1__,1.0,Thousands,O
1,1,VBP,have,NNS,demonstrators,IN,NNS,__START1__,__START1__,Thousands,1.0,of,O
2,2,VBN,marched,VBP,have,NNS,IN,NNS,Thousands,of,1.0,demonstrators,O
3,3,IN,through,VBN,marched,VBP,NNS,IN,of,demonstrators,1.0,have,O
4,4,NNP,London,IN,through,VBN,VBP,NNS,demonstrators,have,1.0,marched,O


In [3]:
# number of sentences
df.sentence_idx.max()

1500.0

In [4]:
# class distribution
df.tag.value_counts(normalize=True )

O        0.852828
B-geo    0.027604
B-gpe    0.020935
B-org    0.020247
I-per    0.017795
B-tim    0.016927
B-per    0.015312
I-org    0.013937
I-geo    0.005383
I-tim    0.004247
B-art    0.001376
I-gpe    0.000837
I-art    0.000748
B-eve    0.000628
I-eve    0.000508
B-nat    0.000449
I-nat    0.000239
Name: tag, dtype: float64

In [5]:
# sentence length
tdf = df.set_index('sentence_idx')
tdf['length'] = df.groupby('sentence_idx').tag.count()
df = tdf.reset_index(drop=False)

In [6]:
# encode categorial variables

le = LabelEncoder()
df['pos'] = le.fit_transform(df.pos)
df['next-pos'] = le.fit_transform(df['next-pos'])
df['next-next-pos'] = le.fit_transform(df['next-next-pos'])
df['prev-pos'] = le.fit_transform(df['prev-pos'])
df['prev-prev-pos'] = le.fit_transform(df['prev-prev-pos'])

In [7]:
df.head()

,sentence_idx,Unnamed: 0,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length
0,1.0,0,18,demonstrators,9,of,18,39,40,__START2__,__START1__,Thousands,O,48
1,1.0,1,33,have,18,demonstrators,9,18,39,__START1__,Thousands,of,O,48
2,1.0,2,32,marched,33,have,18,9,18,Thousands,of,demonstrators,O,48
3,1.0,3,9,through,32,marched,33,18,9,of,demonstrators,have,O,48
4,1.0,4,16,London,9,through,32,33,18,demonstrators,have,marched,O,48


In [8]:
# splitting
y = LabelEncoder().fit_transform(df.tag)

df_train, df_test, y_train, y_test = model_selection.train_test_split(df, y, stratify=y, 
                                                                      test_size=0.25, random_state=SEED, shuffle=True)
print('train', df_train.shape[0])
print('test', df_test.shape[0])

train 50155
test 16719


In [9]:
# some wrappers to work with word2vec
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from collections import defaultdict

   
class Word2VecWrapper(TransformerMixin):
    def __init__(self, window=5,negative=5, size=100, iter=100, is_cbow=False, random_state=SEED):
        self.window_ = window
        self.negative_ = negative
        self.size_ = size
        self.iter_ = iter
        self.is_cbow_ = is_cbow
        self.w2v = None
        self.random_state = random_state
        
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        self.w2v = Word2Vec(sentences_list, 
                            window=self.window_,
                            negative=self.negative_, 
                            size=self.size_, 
                            iter=self.iter_,
                            sg=not self.is_cbow_, seed=self.random_state)

        return self
    
    def has(self, word):
        return word in self.w2v

    def transform(self, X):
        """
        X: a list of words
        """
        if self.w2v is None:
            raise Exception('model not fitted')
        return np.array([self.w2v[w] if w in self.w2v else np.zeros(self.size_) for w in X ])
    


In [10]:
%%time
# here we exploit that word2vec is an unsupervised learning algorithm
# so we can train it on the whole dataset (subject to discussion)

sentences_list = [x.strip() for x in ' '.join(df.word).split('.')]

w2v_cbow = Word2VecWrapper(window=5, negative=5, size=300, iter=300, is_cbow=True, random_state=SEED)
w2v_cbow.fit(sentences_list)

CPU times: user 52.1 s, sys: 553 ms, total: 52.7 s
Wall time: 24.1 s


In [11]:
%%time
# baseline 1 
# random labels
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyClassifier


columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

model = Pipeline([
    ('enc', OneHotEncoder()),
    ('est', DummyClassifier(random_state=SEED)),
])

model.fit(df_train[columns], y_train)

print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))


train 0.05887736725599869
test 0.060439542712750365
CPU times: user 208 ms, sys: 36.9 ms, total: 245 ms
Wall time: 245 ms


In [12]:
%%time
# baseline 2 [GOT]
# pos features + one hot encoding + random forest
from sklearn.preprocessing import OneHotEncoder
from sklearn import ensemble


columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

model = Pipeline([
    ('enc', OneHotEncoder()),
    ('est', ensemble.RandomForestClassifier(random_state=SEED)),
])

model.fit(df_train[columns], y_train)

print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))

train 0.7470525701346687
test 0.5994190204572042
CPU times: user 25.2 s, sys: 121 ms, total: 25.3 s
Wall time: 25.4 s


In [13]:
%%time
# baseline 2 [GOT]
# pos features + one hot encoding + xgboost
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier


columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

model = Pipeline([
    ('enc', OneHotEncoder()),
    ('est', XGBClassifier(random_state=SEED)),
])

model.fit(df_train[columns], y_train)

print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))

train 0.6467439915465034
test 0.42088356317601794
CPU times: user 1min 8s, sys: 206 ms, total: 1min 8s
Wall time: 18.5 s


In [14]:
%%time
# baseline 3 [GOT]
# use word2vec cbow embedding + baseline 2 + RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import scipy.sparse as sp
from sklearn import ensemble

embedding = w2v_cbow
encoder_pos = OneHotEncoder()
X_train = sp.hstack([
    embedding.transform(df_train.word),
    embedding.transform(df_train['next-word']),
    embedding.transform(df_train['next-next-word']),
    embedding.transform(df_train['prev-word']),
    embedding.transform(df_train['prev-prev-word']),
    encoder_pos.fit_transform(df_train[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])
X_test = sp.hstack([
    embedding.transform(df_test.word),
    embedding.transform(df_test['next-word']),
    embedding.transform(df_test['next-next-word']),
    embedding.transform(df_test['prev-word']),
    embedding.transform(df_test['prev-prev-word']),
    encoder_pos.transform(df_test[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])

model = ensemble.RandomForestClassifier(random_state=SEED)
model.fit(X_train, y_train)

print('train', metrics.f1_score(y_train, model.predict(X_train), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(X_test), average='macro'))

train 0.9942251490538192
test 0.8659334953852547
CPU times: user 13min 16s, sys: 4.4 s, total: 13min 20s
Wall time: 13min 20s
